This Utility is the Google Colab version of the one available here:
  - https://github.com/smplusr

This tool contains 3 mains parts:
  - The install mechanism, written in BASH
  - The model setup script, written in Python
  - The actual agent, also written in Python

You only need to run the installer the first time you launch the software.
After that, dependancies will be resolved.

The python scripts will need to be ran in python3. The setup needs to be launched BEFORE the chatbot agent. Once that is done, you can run the agent script.

BE CAREFUL, you might have to do these steps again as soon as you close you python console. Downloads will still persist but the pipeline importation and generation will require to be executed again.

also, GPT-NEO, as one of the most powerful pretrained transformer model that synthetize high quality human native language REQUIRES A REALLY POWERFUL HOST.
The 1.3B version requires at least 11GB of RAM, barely running inside Google Colab free sessions and the 2.7B needing more that 16GB !
CPU and GPU are also really important ! Without some good hardware, simple text generation might take A LOT of time to be completed and it may even crash or damage hardware ! BE CAREFUL !

the GNC-Agent, or GPT-Neo Chatbot Agent is aimed to run on very little function calls. The class comes with 7 prebuild functions, including the init constructor function. List of functions in the source code.

Here is a example of functions that can be ran in a python3 GPT session.

  - agent = GNC_Agent(15)
 
  - agent.Generate(input())

  - print(f"{agent.result}")
  - print(f"{agent.memory}")
  
  - agent.Guard(",max_length = 30")
  - agent.Undo()

  - agent.ReadMemory("file.txt")
  - agent.WriteMemory("file.txt")

The result member of the agent will be the most recent string formated output of the AI. This result can be undone leading to one undo possible on the actual agent's memory. This memory will be built on all the inputs from the user and the AI, leading to actual conversations or context simulation.

There is a built-in debug / sandbox mode allowing to call the agent's inner function from the input prompt. In order to use it, just write a comma: ',' as the first character of a paragraph, then you can unter a method and event parameters. Those are cumulable ! (Example: ",WriteMemory("file.txt") ,Undo()")

Of course, the most important function, from which the debug inputs can be entered is: the Generate method. Though, the Guard (debug method) can be accessed directly.


 -> Huge thanks to EleutherAI and their amazing work on GPT-Neo

In [ ]:
# Install and resolve dependencies

!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

In [ ]:
# Import AI model from gpt-neo

print("Available models:  ")
print("     -> gpt-neo-2.7B")
print("     -> gpt-neo-1.3B")
print("     -> gpt-neo-350M")
print("     -> gpt-neo-125M")
print("")
from transformers import pipelines

model = input("please choose a model: ")
generate = pipelines.pipeline('text-generation', model=f'EleutherAI/{model}')

In [ ]:
# GPT-Neo Chatbot Agent   (GNC-Agent) ~ Uncommented

class GNC_Agent:
    def __init__(self, length):
      self.result = ""
      self.memory = ""
      self.length = 0
      self.max_length = length

    def CountWords(self, input):
      self.length = 0
      word_list = input.split()
      for i in word_list:
        self.length += 1
      return self.length

    def Generate(self, input):
      if self.Guard(input) == True: return

      self.memory += input
      self.CountWords(self.memory)
      
      res = generate(self.memory, max_length = self.length + self.max_length, do_sample = True, temperature = 0.9, pad_token_id = 50256)
      self.result = res[0]["generated_text"]
      
      self.result = self.result.replace(self.memory, "")
      self.result = self.result.replace(input, "")
      self.result = self.result.replace("\n", "")
      
      self.memory += self.result
      return self.result, self.memory

    def Guard(self, input):
      if input[0] == ',':
        input = input.replace(',', 'self.')
        eval(input)
        return True

    def ReadMemory (self, file):
      self.memory = ""
      with open(file, 'r') as content:
        self.memory += content.read()
        content.close()

    def WriteMemory(self, file):
      with open(file, 'w') as content:
        content.write(self.memory)
        content.close()

    def Undo(self):
      self.memory = self.memory.replace(self.result, "")      

In [ ]:
agent = GNC_Agent(15)       # Agent instantiation with base max length set as 15 char.

while(1):
 
  agent.Generate(input())   # Here, Inputs will be retrived from the user and sent to the Generate agent's method.
  print(f"{agent.result}")  # And here, the "result" member of the agent will be printed.